## Data Preprocessing

In [27]:

import numpy as np
import pandas as pd
import math

data = pd.read_csv('titanic.csv')
x = data[["Age", "Fare"]].to_numpy()
y = data["Survived"].to_numpy()
datalen = len(x)
split_percent = 0.10    #10% used for testing data 
split = math.floor(datalen*split_percent)

#no need to shuffle, data already randomized in the .csv file
x_test = x[:split].T
x_train = x[split:].T
y_test = y[:split]
y_train = y[split:]

#print(x_test)


## 

In [29]:

#hyperparams
epochs = 10
learning_rate = 0.01

#init our weight vector based on how many features we have in the data
weights : "np.ndarray[np.floating]" = np.random.normal(size=x.shape[1])
bias : "np.floating" = np.random.normal()

def sigmoid(X):
    return 1/(1 + np.exp(-X))

def forward(X, weights, bias):
    return sigmoid(np.dot(weights.T, X) + bias)

def loss(Y, A):
    return -(Y * np.log(A) + (1 - Y) * np.log(1 - A))

def cost(Y, A):
    return -1/len(A) * np.sum(loss(Y, A))

#partial derivitive of cost with respect to weights
def dCostWRTw(X, A, Y):
    return np.dot(X, (A-Y).T)/len(A)

#partial derivitive of cost with respect to bias
def dCostWRTb(A, Y):
    return np.sum(A-Y)/len(A)


#training loop
for epoch in range(epochs):
    A = forward(x_train, weights, bias)
    c = cost(y_train, A)
    print("Epoch: " + str(epoch) + ", Cost: " + str(c))
    dw = dCostWRTw(x_train, A, y_train)
    db = dCostWRTb(A, y_train)
    weights = weights - learning_rate * dw
    bias = bias - learning_rate * db




Epoch: 0, Cost: -29.30941124844555
Epoch: 1, Cost: -24.468695789686333
Epoch: 2, Cost: -19.627982430618157
Epoch: 3, Cost: -14.787289873332464
Epoch: 4, Cost: -9.946914969504398
Epoch: 5, Cost: -5.1168324318294856
Epoch: 6, Cost: -1.0881186228865858
Epoch: 7, Cost: -0.9222383659176796
Epoch: 8, Cost: -1.558910328959825
Epoch: 9, Cost: nan


/tmp/ipykernel_3048205/1849891159.py:16: RuntimeWarning: divide by zero encountered in log
  return -(Y * np.log(A) + (1 - Y) * np.log(1 - A))
/tmp/ipykernel_3048205/1849891159.py:16: RuntimeWarning: invalid value encountered in multiply
  return -(Y * np.log(A) + (1 - Y) * np.log(1 - A))
